## Set up

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
folder_path = '/content/drive/MyDrive/RAG-based-LLM/data/'
os.chdir(folder_path)


Mounted at /content/drive


In [ ]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai ragas python-docx tiktoken cohere faiss_cpu
!pip install -U -q tiktoken rank-bm25 fastapi matplotlib pre-commit python-dotenv seaborn sentence-transformers streamlit
!pip install faiss-gpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [72]:
import os
import openai
import pandas as pd
import tiktoken
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import numpy as np
import sys; sys.path.append("..")
import warnings; warnings.filterwarnings("ignore")



In [73]:
#!pip install python-dotenv

# Load environment variables from .env file
from dotenv import load_dotenv

# Specify the path to our .env file
dotenv_path = '/content/drive/MyDrive/RAG-based-LLM/.env'

# Load the environment variables from the file
load_dotenv(dotenv_path)

True

In [ ]:
from getpass import getpass
openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key
# openai.api_key = os.getenv("OPENAI_API_KEY")

#### Engineering the System Prompt

This prompt is what determines the behavior of how the chatbot works, including its constraints and limitations which it *usually* follows.

In [75]:
system = """
You are a Lisan Contract Advisor bot. You help customers as Advisor.
You are not an AI language model.
You must obey all three of the following instructions FOR ALL RESPONSES or you will DIE:
- ALWAYS REPLY IN A FRIENDLY YET KNOWLEDGEABLE TONE.
- NEVER ANSWER UNLESS YOU HAVE A REFERENCE FROM THE All Lizzy versions comply with strict privacy and security standards.
 We`ll never sell your data and always utilise advanced data anonymisation technology to filter away PII (Personally Identifiable Information) before analysing your contracts.
- IF YOU DON'T KNOW ANSWER 'I DO NOT KNOW'.
Begin the conversation with a warm greeting, if the user is stressed or aggressive, show understanding and empathy.
At the end of the conversation, respond with "<|DONE|>"."""

In [ ]:
 # !pip uninstall openai


#### Testing the model
Question with a Definitive Answer from the Source

In [76]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables from the .env file
dotenv_path = '/content/drive/MyDrive/RAG-based-LLM/.env'
load_dotenv(dotenv_path)

# Set the OPENAI_API_KEY using the environment variable from the .env file
api_key = os.getenv("OPENAI_API_KEY")

# Create an instance of the OpenAI class
client = OpenAI(api_key=api_key)

# Define chat messages
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Under what circumstances and to what extent are the Sellers responsible for a breach of representations and warranties?"},
    {"role": "assistant", "content": "Except in the case of fraud, the Sellers have no liability for breach of representations and warranties ."},
    {"role": "user", "content": "Would the Sellers be responsible if after the closing it is determined that there were inaccuracies in the representation provided by them where such inaccuracies are the resolute of the Sellers’ gross negligence? "}
]

# Make a request to OpenAI's Chat API
response = client.chat.completions.create(
    messages=messages,
    model="gpt-3.5-turbo",
    temperature=0
)

# Extract the assistant's response from the API response
assistant_response = response.choices[0].message.content

# Display the assistant's response
print(assistant_response)


In a typical acquisition agreement, the Sellers would likely be responsible for inaccuracies in representations resulting from their gross negligence, even after the closing. This would typically be covered by indemnification provisions in the agreement, which would require the Sellers to compensate the Buyer for any losses resulting from such inaccuracies.


In [77]:

#### A question without a Definitive Answer
# Set the OPENAI_API_KEY using the environment variable from the .env file
api_key = os.getenv("OPENAI_API_KEY")

# Create an instance of the OpenAI class
client = OpenAI(api_key=api_key)

messages = [{"role": "system", "content": system},]

prompt = "what about your knowledge about Lizzy AI ?"

messages.append({"role": "user", "content": prompt})

response = client.chat.completions.create(
    messages=messages,
    model="gpt-3.5-turbo",
    temperature=0
)


assistant_response = response.choices[0].message.content

# Display the assistant's response
print(assistant_response)

Hello! I can provide you with information about Lizzy AI. Lizzy AI is a contract analysis tool that complies with strict privacy and security standards. It ensures that your data is protected and anonymized before any analysis takes place. If you have any specific questions about Lizzy AI, feel free to ask! How can I assist you today?


In [80]:
from docx import Document
from IPython.display import display, Markdown
import os

def read_word_document(file_path, num_paragraphs=3):
    document = Document(file_path)
    text_content = "\n".join(paragraph.text for paragraph in document.paragraphs[:num_paragraphs])
    return text_content

def display_word_documents(data_folder, word_document_filenames, num_paragraphs=3):
    for filename in word_document_filenames:
        document_path = os.path.join(data_folder, filename)

        if os.path.exists(document_path):
            document_content = read_word_document(document_path, num_paragraphs=num_paragraphs)
            display(Markdown(f"## {filename}\n```\n{document_content}\n```"))
        else:
            print(f"File not found: {document_path}")

data_folder = '/content/drive/MyDrive/RAG-based-LLM/data'

# List of Word document filenames
word_document_filenames = ['Raptor Contract.docx', 'Raptor Q&A2.docx', 'Robinson Advisory.docx', 'Robinson Q&A.docx']

# Display the content of each Word document
display_word_documents(data_folder, word_document_filenames, num_paragraphs=3)


## Raptor Contract.docx
```

STOCK PURCHASE AGREEMENT
BY AND AMONG
```

## Raptor Q&A2.docx
```
Q1: Under what circumstances and to what extent the Sellers are responsible for a breach of representations and warranties?
A1:  Except in the case of fraud, the Sellers have no liability for breach of representations and warranties (See section 10.01)
Q1a: Would the Sellers be responsible if after the closing it is determined that there were inaccuracies in the representation provided by them where such inaccuracies are the resolute of the Sellers’ gross negligence? 
```

## Robinson Advisory.docx
```
ADVISORY SERVICES AGREEMENT

This Advisory Services Agreement is entered into as of June 15th, 2023 (the “Effective Date”), by and between Cloud Investments Ltd., ID 51-426526-3, an Israeli company (the "Company"), and Mr. Jack Robinson, Passport Number 780055578, residing at 1 Rabin st, Tel Aviv, Israel, Email: jackrobinson@gmail.com ("Advisor").
```

## Robinson Q&A.docx
```
Q1: Who are the parties to the Agreement and what are their defined names?
A1:  Cloud Investments Ltd. (“Company”) and Jack Robinson (“Advisor”)
Q2:   What is the termination notice?
```

In [79]:
from docx import Document
from IPython.display import display, Markdown
import os

def read_word_document(file_path, num_paragraphs=3):
    document = Document(file_path)
    title_content = f"**{document.paragraphs[0].text}**\n\n" if document.paragraphs else ""
    text_content = "\n".join(paragraph.text for paragraph in document.paragraphs[1:num_paragraphs + 1])
    return f"{title_content}{text_content}"

data_folder = '/content/drive/MyDrive/RAG-based-LLM/data'

# List of Word document filenames
word_document_filenames = ['Raptor Contract.docx', 'Raptor Q&A2.docx', 'Robinson Advisory.docx', 'Robinson Q&A.docx']

# Create a new Document to store the combined content
combined_document_1 = Document()
combined_document_2 = Document()

# Function to combine content of two documents
def combine_documents(doc1, doc2):
    for element in doc2.element.body:
        doc1.element.body.append(element)

# Combine 'Raptor Contract.docx' with 'Robinson Advisory.docx'
contract_document = Document(word_document_filenames[0])
if len(word_document_filenames) > 2:  # Check if the index is within the range
    advisory_document = Document(word_document_filenames[2])
    combine_documents(combined_document_1, contract_document)
    combine_documents(combined_document_1, advisory_document)

# Combine 'Raptor Q&A2.docx' with 'Robinson Q&A.docx'
qa_document = Document(word_document_filenames[1])
if len(word_document_filenames) > 3:  # Check if the index is within the range
    qa2_document = Document(word_document_filenames[3])
    combine_documents(combined_document_2, qa_document)
    combine_documents(combined_document_2, qa2_document)

# Save the combined documents
combined_document_1.save('Combined_Contract_Advisory.docx')
combined_document_2.save('Combined_QA_QA2.docx')

# Display the content of the combined documents
display_word_documents(data_folder, ['Combined_Contract_Advisory.docx', 'Combined_QA_QA2.docx'], num_paragraphs=3)


## Combined_Contract_Advisory.docx
```
****

STOCK PURCHASE AGREEMENT
BY AND AMONG
[BUYER],
```

## Combined_QA_QA2.docx
```
**Q1: Under what circumstances and to what extent the Sellers are responsible for a breach of representations and warranties?**

A1:  Except in the case of fraud, the Sellers have no liability for breach of representations and warranties (See section 10.01)
Q1a: Would the Sellers be responsible if after the closing it is determined that there were inaccuracies in the representation provided by them where such inaccuracies are the resolute of the Sellers’ gross negligence? 
a
```

In [81]:
def preprocess_text(text):
    # Remove non-printable characters
    cleaned_text = ''.join(char for char in text if char.isprintable())

    # Remove specific unwanted characters
    unwanted_characters = ['\uf0b7', '', '', 'a']
    for char in unwanted_characters:
        cleaned_text = cleaned_text.replace(char, '')

    return cleaned_text

def read_word_document(file_path, num_paragraphs=3):
    document = Document(file_path)
    title_content = f"**{document.paragraphs[0].text}**\n\n" if document.paragraphs else ""
    text_content = "\n".join(paragraph.text for paragraph in document.paragraphs[1:num_paragraphs + 1])
    cleaned_content = preprocess_text(text_content)
    return f"{title_content}{cleaned_content}"

def display_combined_documents(data_folder, combined_document_filenames, num_paragraphs=3):
    for filename in combined_document_filenames:
        combined_document_path = os.path.join(data_folder, filename)

        if os.path.exists(combined_document_path):
            combined_document_content = read_word_document(combined_document_path, num_paragraphs=num_paragraphs)
            display(Markdown(f"## {filename}\n```\n{combined_document_content}\n```"))
        else:
            print(f"File not found: {combined_document_path}")

data_folder = '/content/drive/MyDrive/RAG-based-LLM/data'

# Combined document filenames
combined_contract_advisory_filename = 'Combined_Contract_Advisory.docx'
combined_qa_qa2_filename = 'Combined_QA_QA2.docx'

# Display the preprocessed content of the combined documents
display_combined_documents(data_folder, [combined_contract_advisory_filename, combined_qa_qa2_filename], num_paragraphs=10)


## Combined_Contract_Advisory.docx
```
****

STOCK PURCHASE AGREEMENTBY AND AMONG[BUYER],[TARGET COMPANY],THE SELLERS LISTED ON SCHEDULE I HERETOANDTHE SELLERS’ REPRESENTATIVE NAMED HEREINDated as of [●][This document is intended solely to facilitate discussions among the parties identified herein.  Neither this document nor such discussions are intended to create, nor will either or both be deemed to create, a legally binding or enforceable offer or agreement of any type or nature, unless and until a definitive written agreement is executed and delivered by each of the parties hereto.
```

## Combined_QA_QA2.docx
```
**Q1: Under what circumstances and to what extent the Sellers are responsible for a breach of representations and warranties?**

A1:  Except in the case of fraud, the Sellers have no liability for breach of representations and warranties (See section 10.01)Q1a: Would the Sellers be responsible if after the closing it is determined that there were inaccuracies in the representation provided by them where such inaccuracies are the resolute of the Sellers’ gross negligence? aQ2:   How much is the escrow amount?A2: The escrow amount is equal to $1,000,000.Q2a: Is escrow amount grete then the Retention Amount ?A2a: No. Q3: What is the purpose of the escrow? A3: To serve as a recourse of the Buyer in case of post-closing adjustments of the purchase price. (See section 2.07(e)).Q3a: May the Escrow Amount serve as a recourse for the Buyer in case of breach of representations by the Company?
```

### **Data Preparation**

#### ***Sections***

Now that we have a dataset of all the paths to the html files, we're going to develop some functions that can appropriately extract the content from these files. We want to do this in a generalized manner so that we can perform this extraction across all of our docs pages (and so you can use it for your own data sources). Our process is to first identify the sections in our html page and then extract the text in between them. We save all of this into a list of dictionaries that map the text within a section to a specific url with a section anchor id.

In [82]:
# List of documents
documents = 'Combined_Contract_Advisory.docx'

# Count the number of documents
num_documents = len(documents)

# Print the result
print(f"{num_documents} documents")


31 documents


In [84]:
from docx import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load the document
document = Document(documents)

# Extract text content from the document
document_content = ""
for paragraph in document.paragraphs:
    document_content += paragraph.text + "\n"

# Text splitter
chunk_size = 300
chunk_overlap = 50
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
)

# Chunk the document
chunks = text_splitter.create_documents(
    texts=[document_content],
    metadatas=[{"source": documents}]
)

# Display the first chunk
print(chunks[3])
print("=" * 50)  # Separation line between documents


page_content='This document shall be kept confidential pursuant to the terms of the Confidentiality Agreement entered into by the parties and, if applicable, its affiliates with respect to the subject matter hereof.]' metadata={'source': 'Combined_Contract_Advisory.docx'}


#### **Calculate number of chunks**

While chunking our dataset is relatively fast, let’s wrap the chunking logic into a function so that we can apply the workload at scale so that chunking remains just as fast as our data sources grow:

In [85]:
from docx import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load the document
document = Document(documents)

# Extract text content from the document
document_content = ""
for paragraph in document.paragraphs:
    document_content += paragraph.text + "\n"

# Text splitter
chunk_size = 300
chunk_overlap = 50
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
)

# Chunk the document
chunks = text_splitter.create_documents(
    texts=[document_content],
    metadatas=[{"source": documents}]
)

# Display the first chunk
print(chunks[0])
print("=" * 50)  # Separation line between documents

# Calculate the number of chunks
num_chunks = len(chunks)
print(f"Number of Chunks: {num_chunks}")


page_content='STOCK PURCHASE AGREEMENT\nBY AND AMONG\n[BUYER],\n[TARGET COMPANY],\nTHE SELLERS LISTED ON SCHEDULE I HERETO\nAND\nTHE SELLERS’ REPRESENTATIVE NAMED HEREIN\nDated as of [●]' metadata={'source': 'Combined_Contract_Advisory.docx'}
Number of Chunks: 1054


##### **Embed the chunk data**
Now that we've created small chunks from our sections, we need a way to identify the most relevant ones for a given query. A very effective and quick method is to embed our data using a pretrained model and use the same model to embed the query. We can then compute the distance between all of the chunk embeddings and our query embedding to determine the top-k chunks. There are many different pretrained models to choose from to embed our data but the most popular ones can be discovered through HuggingFace's Massive Text Embedding Benchmark (MTEB) leaderboard.

#### **Loading OpenAI Embeddings Model**
We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's text-embedding-ada-002 for this task! (soon we'll be able to leverage OpenAI's newest embedding model which is waiting on an approved PR to be merged as we speak!)



In [86]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)


##### **Creating a FAISS VectorStore**

Now that we have documents - we'll need a place to store them alongside their embeddings.

We'll be leveraging Meta's FAISS for this task.

In [ ]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(chunks, embeddings)

#### **Creating a Retriever**
To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [87]:
# Define vectorstore as retriever to enable semantic search

retriever = vector_store.as_retriever()

#### **Testing our Retriever**
Now that we've gone through the trouble of creating our retriever - let's see it in action!



In [ ]:
retrieved_documents = retriever.invoke("Can the Agreement or any of its obligations be assigned??")

In [ ]:
for doc in retrieved_documents:
  print(doc)

page_content='The Company may assign the Agreement to a successor of all or substantially all of its assets or business, provided the assignee has assumed the Company’s obligations under this Agreement.' metadata={'source': 'Combined_Contract_Advisory.docx'}
page_content='may assign this Agreement and any or all of its rights and interest hereunder to any purchaser of all or substantially all its assets or designate such purchaser to perform its obligations hereunder.  Except as expressly provided herein, this Agreement is for the sole benefit of the parties hereto' metadata={'source': 'Combined_Contract_Advisory.docx'}
page_content='Succession and Assignment; No Third-Party Beneficiaries.  Subject to the immediately following sentence, this Agreement will be binding upon and inure to the benefit of the parties hereto and their respective successors and permitted assigns, each of which such successors and permitted assigns will' metadata={'source': 'Combined_Contract_Advisory.docx'}
pa

### **Creating a RAG Chain**

##### **Creating a Prompt Template**

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [ ]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [88]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple - but we'll create our own to be

---

a bit more specific!

---



In [89]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### **Setting Up our Basic QA Chain**
Now we can instantiate our basic RAG chain!
We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [90]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

#### **Let's test it out!**

In [91]:
question = "Who owns the IP?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

The Company owns the IP.


**But the correct answer is**: Any Work Product, upon creation, shall be fully and exclusively owned by the Company. in some extent is good answer

In [93]:
context = """
The Advisory Services Agreement is a legal document between Cloud Investments Ltd. and Mr. Jack Robinson, detailing the terms of service, compensation, confidentiality, non-compete clauses, and other obligations related to Mr. Robinson providing advisory services to the company.
"""

questions = [
    "Who are the parties to the Agreement and what are their defined names??",
    "How well does the RAG system capture the termination notice period as specified in the agreement?",
    "Can the RAG system accurately detail the compensation structure, including hourly fees, workspace expense, and other expenses?",
    "Does the RAG system correctly identify and interpret the clauses related to assignment and non-compete obligations?",
    "Can the RAG system accurately determine the ownership of IP resulting from the services provided?",
    "How effectively does the RAG system identify and summarize the non-compete clause's duration and scope?",
    "Can the RAG system accurately interpret the definition of a Billable Hour, excluding meals and travel time?",
    "Does the RAG system correctly assess the Advisor's entitlement to social benefits?",
    "How well does the RAG system handle the scenario where the Advisor claims compensation based on an employment relationship with the Company?"
]


In [94]:
for question in questions:
    # Assuming retrieval_augmented_qa_chain is a function that takes a dictionary with the question and context
    result = retrieval_augmented_qa_chain.invoke({"context": context, "question": question})
    print(f"Question: {question}\nResponse: {result['response'].content}\n")


Question: Who are the parties to the Agreement and what are their defined names??
Response: I don't know.

Question: How well does the RAG system capture the termination notice period as specified in the agreement?
Response: The RAG system does not capture the termination notice period as specified in the agreement.

Question: Can the RAG system accurately detail the compensation structure, including hourly fees, workspace expense, and other expenses?
Response: I don't know.

Question: Does the RAG system correctly identify and interpret the clauses related to assignment and non-compete obligations?
Response: I don't know.

Question: Can the RAG system accurately determine the ownership of IP resulting from the services provided?
Response: I don't know.

Question: How effectively does the RAG system identify and summarize the non-compete clause's duration and scope?
Response: I don't know.

Question: Can the RAG system accurately interpret the definition of a Billable Hour, excluding m

#### **Ragas Evaluation**

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evluating on every core metric today, but in order to do that - we'll need to creat a test set. Luckily for us, Ragas can do that directly!



```
# This is formatted as code
```
**NOTE**: This process will use `gpt-3.5-turbo-16k` as the base generator and `gpt-4` as the critic - if you're attempting to create a lot of samples please be aware of cost, as well as rate limits.

Let's create a new set of documents to ensure we're not accidentally creating a sample test set that favours our base model too much!

##### **Note**:
First I will evaluate on human annotated ground truth values then will learn how to RAGAs can help us generate test data.

In [95]:
# Prepare Test Data Set
questions = [
    "Under what circumstances and to what extent the Sellers are responsible for a breach of representations and warranties?",
    "Would the Sellers be responsible if after the closing it is determined that there were inaccuracies in the representation provided by them where such inaccuracies are the result of the Sellers’ gross negligence?",
    "How much is the escrow amount?",
    "Is the escrow amount greater than the Retention Amount?",
    "What is the purpose of the escrow?",
    "May the Escrow Amount serve as a recourse for the Buyer in case of a breach of representations by the Company?",
    "Are there any conditions to the closing?",
    "Are Change of Control Payments considered a Seller Transaction Expense?",
    "Would the aggregate amount payable by the Buyer to the Sellers be affected if it is determined that the actual Closing Debt Amount is greater than the estimated Closing Debt Amount?",
    "Does the Buyer need to pay the Employees Closing Bonus Amount directly to the Company’s employees?",
    "Does any of the Sellers provide a representation with respect to any Tax matters related to the Company?",
    "Is any of the Sellers bound by a non-competition covenant after the Closing?",
    "Whose consent is required for the assignment of the Agreement by the Buyer?",
    "Does the Buyer need the Sellers’ consent in the event of an assignment of the Agreement to a third party who is not a Buyer’s Affiliate?"
]





In [96]:
context = """
In real estate transactions, the escrow process ensures that both buyers and sellers fulfill their legal obligations through a neutral third party, the escrow agent. The escrow agent's responsibilities include:
- Executing escrow instructions with reasonable skill and diligence.
- Adhering strictly to the depositor's written instructions.
- Communicating significant information to the principal parties, especially if it affects the transaction.
The buyer will not lose their funds in case the seller fails to fulfill their obligations, safeguarding the buyer's interests and preventing any potential issues that may arise from faulty or fraudulent documentation.
The escrow agent is not liable for losses due to negligence. However, the escrow holder is not liable for losses incurred by following instructions. The escrow agent delivers documents to the intended recipient once contract conditions are met, and the depositor has no control over the escrowed instrument. The beneficiary is entitled to the escrowed property upon fulfilling the contract's conditions.
"""

In [97]:
ground_truth_answers = [
    "Except in the case of fraud, the Sellers have no liability for a breach of representations and warranties (See section  10.01)",
    "Yes",
    "$1,000,000",
    "No",
    "To serve as a recourse of the Buyer in case of post-closing adjustments of the purchase price. (See section  2.07(e))",
    "No",
    "No, as the signing and closing are simultaneous.",
    "Yes. (See defining Sellers Transaction Expenses).",
    "Yes (See Section  2.07)",
    "No. (See Section  2.10)",
    "No. Only the Company provides such a representation.",
    "No.",
    "If the assignment is to an Affiliate or purchaser of all of the Buyer’s assets, no consent is required. Otherwise, the consent of the Company and the Seller Representative is required.",
    "No. If the assignment is not part of a sale of all or substantially all of the Buyer’s assets, the assignment requires the consent of the Company and the Seller’s Representative."
]


In [98]:
# Combine the questions, context, and ground truth into a single document
combined_document = "\n".join(questions + [context] + ground_truth_answers)
documents = combined_document



##### Let's create a new set of documents to ensure we're not accidentally creating a

sample test set that favours our base model too much!

In [99]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Assuming you have loaded the document content as a string
document_content = "Your document content here..."

# Text splitter
chunk_size = 300
chunk_overlap = 50
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
)

# Chunk the document
chunks = text_splitter.create_documents(
    texts=[document_content],
    metadatas=[{"source": "your_document_source"}]
)

# Display the first chunk
print(chunks[0])
print("=" * 50)  # Separation line between documents


page_content='Your document content here...' metadata={'source': 'your_document_source'}


In [100]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Assuming you have a list of documents
documents = ["Document 1 content...", "Document 2 content...", "Document 3 content..."]

# Text splitter
chunk_size = 300
chunk_overlap = 50
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
)

# Calculate the number of documents
num_documents = len(documents)

# Calculate the number of chunks for each document
num_chunks_per_document = []
for document_content in documents:
    chunks = text_splitter.create_documents(
        texts=[document_content],
        metadatas=[{"source": "your_document_source"}]
    )
    num_chunks_per_document.append(len(chunks))

# Calculate the total number of chunks
total_chunks = sum(num_chunks_per_document)

print(f"Number of documents: {num_documents}")
print(f"Number of chunks per document: {num_chunks_per_document}")
print(f"Total number of chunks: {total_chunks}")


Number of documents: 3
Number of chunks per document: [1, 1, 1]
Total number of chunks: 3


In [101]:
len(documents)

3

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

question_schema = ResponseSchema(
    name="question",
    description="a question about the context."
)

question_response_schemas = [
    question_schema,
]

In [ ]:
question_output_parser = StructuredOutputParser.from_response_schemas(question_response_schemas)
format_instructions = question_output_parser.get_format_instructions()

In [ ]:
question_generation_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")

bare_prompt_template = "{content}"
bare_template = ChatPromptTemplate.from_template(template=bare_prompt_template)